# **SSIM Frame Similarity - Detailed Process Explanation**

# **Step 1: Import Libraries**
# Import required libraries for video processing (OpenCV), memory and CPU profiling (psutil), and structural similarity computation (SSIM from skimage).
# These libraries are essential for video frame comparison, performance monitoring, and efficient memory management during video processing.

# **Step 2: SSIM Frame Similarity Function**
# A function to compute the Structural Similarity Index (SSIM) score between two video frames.
# SSIM measures perceptual similarity, which is useful for deciding whether two frames are similar enough to skip processing.

# **Step 3: Video Frame Extraction**
# Function to extract and convert video frames into grayscale.
# The frames are read from the input video, stored in a list, and then used for SSIM-based frame comparison.

# **Step 4: Video Frame Loading and Time Calculation**
# Load video frames and measure the time it takes for this process.
# The function `get_video_frames` returns a list of grayscale frames which are necessary for comparison.
# The time to load all frames from the video is recorded to evaluate performance.

# **Step 5: Video Writer Setup**
# A video writer is initialized to save the processed video.
# Set the codec and output video file format (`.mp4`), and extract the frame size and frame rate from the first frame.

# **Step 6: Frame Processing and Comparison**
# Loop through each frame of the video starting from frame 1.
# Compare each current frame with the last processed frame using the SSIM score.
# If SSIM score is high (indicating high similarity), the frame is skipped; otherwise, it is processed and written to the output video.

# **Step 7: CPU Usage Tracking**
# Track CPU usage for each frame comparison during video processing.
# The psutil library is used to record CPU usage, which is stored for later calculation of average CPU usage during the entire processing period.

# **Step 8: Metrics Collection**
# After processing, calculate and log the following:
#  - Total frames processed and skipped
#  - Frame processing time (loading, processing, and total)
#  - Memory consumption before and after processing (using psutil)
#  - Average CPU usage during processing
# These metrics help assess the performance and efficiency of the video processing pipeline.

# **Step 9: Final Results and Logging**
# Print detailed statistics regarding the processing of the video, including the number of frames processed, skipped, and the percentage of skipped frames.
# Additionally, log time-related details such as processing FPS, total time, and memory consumption during processing to evaluate efficiency.

# **Step 10: Video Writing and Resource Cleanup**
# The processed video is saved to the specified output path.
# Release the video writer after finishing processing and writing the frames.


In [ ]:
# Dry run

import cv2
import numpy as np
import time
import psutil  # For memory and CPU profiling
from skimage.metrics import structural_similarity as ssim  # Import SSIM from skimage

# Set the video path (update this to your video file path)
video_path = '/content/input 1.mov'  # Change this to your video file path
output_video_path = '/content/Output_SSIM.mp4'  # Output video path

# Function to calculate the SSIM between two frames
def calculate_ssim(frame1, frame2):
    # Calculate SSIM
    score, _ = ssim(frame1, frame2, full=True)
    return score

# Read video frames
def get_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        frames.append(gray_frame)
    cap.release()
    return frames

# Load video frames
start_time = time.time()  # Start time for reading frames
frames = get_video_frames(video_path)
loading_time = time.time() - start_time  # Time taken to load frames

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
fps = 30  # Set to your video's frame rate
frame_height, frame_width = frames[0].shape
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process frames and write to output video
frame_count = len(frames)
skipped_frames = 0
processed_frames = 0
out.write(cv2.cvtColor(frames[0], cv2.COLOR_GRAY2BGR))  # Write the first frame
last_processed_frame_index = 0  # Track the last processed frame index

# Start time for processing and initialize CPU usage tracking
start_processing_time = time.time()
cpu_usage_samples = []

# Start comparing from frame 1 onwards
for i in range(1, frame_count):
    # Compare current frame with the last processed frame
    current_frame = frames[i]
    last_frame = frames[last_processed_frame_index]

    ssim_score = calculate_ssim(last_frame, current_frame)
    # Record CPU usage for the current comparison
    cpu_usage_samples.append(psutil.cpu_percent(interval=None))
    print(f"Comparing Frame {last_processed_frame_index} with Frame {i}: SSIM = {ssim_score:.4f}")

    if ssim_score >= 0.75:  # If SSIM score is high, skip the current frame
        skipped_frames += 1
    else:  # If SSIM score is low, process the current frame
        out.write(cv2.cvtColor(current_frame, cv2.COLOR_GRAY2BGR))  # Write the current frame if not similar
        last_processed_frame_index = i  # Update the last processed frame index to the current frame
        processed_frames += 1  # Count the processed frame

# Calculate processing time
end_processing_time = time.time()
processing_time = end_processing_time - start_processing_time
total_time = loading_time + processing_time  # Total time includes loading and processing

# Release the video writer
out.release()

# Memory usage before and after processing
process = psutil.Process()
memory_info = process.memory_info()
initial_memory = memory_info.rss / (1024 * 1024)  # Convert bytes to MB
final_memory = (memory_info.rss + (skipped_frames * frames[0].nbytes)) / (1024 * 1024)  # Rough estimate of final memory
memory_consumed = final_memory - initial_memory

# Calculate average CPU usage during processing
average_cpu_usage = np.mean(cpu_usage_samples)

# Log the required information
print(f"Processed video saved as: {output_video_path}")
print(f"Total number of frames: {frame_count}")
print(f"Number of frames skipped: {skipped_frames}")
print(f"Number of frames processed: {processed_frames}")
print(f"% of frames skipped: {(skipped_frames / frame_count) * 100:.2f}%")
print(f"Frames processing per second: {frame_count / processing_time:.2f} FPS")
print(f"Total time taken to load frames: {loading_time:.6f} seconds")
print(f"Total processing time: {processing_time:.6f} seconds")
print(f"Total time (loading + processing): {total_time:.6f} seconds")
print(f"Average time per frame comparison: {processing_time / (frame_count - 1):.6f} seconds")
print(f"Total memory usage: Initial = {initial_memory:.2f} MB, Final = {final_memory:.2f} MB")
print(f"Memory consumed during processing: {memory_consumed:.2f} MB")
print(f"Average CPU usage during processing: {average_cpu_usage:.2f}%")
print(f"Total processing load: {frame_count} frames")


Comparing Frame 0 with Frame 1: SSIM = 0.6387
Comparing Frame 1 with Frame 2: SSIM = 0.6485
Comparing Frame 2 with Frame 3: SSIM = 0.6398
Comparing Frame 3 with Frame 4: SSIM = 0.6276
Comparing Frame 4 with Frame 5: SSIM = 0.6400
Comparing Frame 5 with Frame 6: SSIM = 0.7784
Comparing Frame 5 with Frame 7: SSIM = 0.6590
Comparing Frame 7 with Frame 8: SSIM = 0.7831
Comparing Frame 7 with Frame 9: SSIM = 0.6707
Comparing Frame 9 with Frame 10: SSIM = 0.7782
Comparing Frame 9 with Frame 11: SSIM = 0.6772
Comparing Frame 11 with Frame 12: SSIM = 0.7975
Comparing Frame 11 with Frame 13: SSIM = 0.6737
Comparing Frame 13 with Frame 14: SSIM = 0.7681
Comparing Frame 13 with Frame 15: SSIM = 0.6276
Comparing Frame 15 with Frame 16: SSIM = 0.7383
Comparing Frame 16 with Frame 17: SSIM = 0.7351
Comparing Frame 17 with Frame 18: SSIM = 0.6984
Comparing Frame 18 with Frame 19: SSIM = 0.7144
Comparing Frame 19 with Frame 20: SSIM = 0.7097
Comparing Frame 20 with Frame 21: SSIM = 0.6899
Comparing Fr